This is the smart factory exercise


In [1]:
import mdptoolbox, mdptoolbox.example
import numpy as np

In [30]:
# old example for mdp
# Definition of the three different transition probability matrices
tpmLazy = np.array([[0, 0.95, 0.04, 0.01],[0, 1, 0, 0],[0, 0.2, 0.8, 0],[0, 0, 0.8, 0.2]])
tpmDiligent = np.array([[0, 0.95, 0.04, 0.01],[0, 0.3, 0.7, 0],[0, 0.2, 0.8, 0], [0, 0, 0.8, 0.2]])
tpmEDiligent = np.array([[0, 0.95, 0.04, 0.01],[0, 0.1, 0.2, 0.7], [0, 0, 0.7, 0.3],[0, 0, 0.8, 0.2]])

# This is an example how the construction of the reward matrix is not working as input for the mdp
rewardLazy = np.array([0, 1, 2, 4])
rewardDiligent = np.array([0, -5, 2, 4])
rewardEDiligent = np.array([0, -20, -10, 4])

# This is an example how to build the reward matrix 
rewardfull = np.array([[0, 0, 0],[1, -1, -10], [2, 2, -1], [10, 10, 10]])
print(rewardfull.shape)

# Bring all the tpm together
tmpFull = np.array([tpmLazy, tpmDiligent, tpmEDiligent])
print(tmpFull.shape)

# Definition of the mdp with discount factor, maximal iterations, the tranisition probability matrix and the reward matrix
mdpresultPolicy = mdptoolbox.mdp.PolicyIteration(tmpFull,rewardfull,0.3, max_iter=100)
mdpresultValue = mdptoolbox.mdp.ValueIteration(tmpFull,rewardfull,0.3, max_iter=100)

# Run the MDP
mdpresultPolicy.run()
mdpresultValue.run()

"""-------- HERE ARE THE SOLUTIONS ----------------"""

print('PolicyIteration:')
print(mdpresultPolicy.policy)
print(mdpresultPolicy.V)
print(mdpresultPolicy.iter)

print('ValueIteration:')
print(mdpresultValue.policy)
print(mdpresultValue.V)
print(mdpresultValue.iter)

(4, 3)
(3, 4, 4)
PolicyIteration:
(0, 0, 0, 0)
(0.4740921452567589, 1.4285714285714286, 2.7443609022556394, 11.338985762278037)
1
ValueIteration:
(0, 0, 0, 0)
(0.4622244, 1.417, 2.728424, 11.321488)
4


Hints: define states (1572864) + distance influences reward
goal: minimize distance 
items: white blue red

create a warehouse of size 3x3, with a separate start 
-> first move of the agent is always to 1x1

MDP 4-tuple(State, Actions, Probability Transition matrix P_a(s,s'), Reward matrix R_a(s,s'))

possible States: <br>
start/end point: <br>
agent start, agent end, empty, white, blue, red => 6 states <br>
storage fields: <br> 
empty, white, blue, red with <br>
=> 4 states, but what is with the agent? <br>
would make the 1572864 states

In [8]:
# Transition matrix for 2x2 warehouse + start position:
# only one direction
#tmpStart = np.array([[0, 1, 0, 0, 0],[1, 0, 0, 0, 0]])
#tmpOne = np.array([[0, 0, 1, 0, 0],[0, 0, 0, 1, 0]])
#tmpTwo = np.array([[0, 0, 0, 0, 1],[0, 0, 1, 0, 0]])
#tmpThree = np.array([[0, 0, 0, 0, 1],[0, 0, 0, 1, 0]])
#tmpFour = np.array([[1, 0, 0, 0, 0],[0, 0, 0, 0, 1]])
# arrange shape differently
tmpA = np.array([[0, 1, 0, 0, 0],[0, 0, 1, 0, 0],[0, 0, 0, 0, 1],[0, 0, 0, 0, 1],[1, 0, 0, 0, 0]])
tmpB = np.array([[1, 0, 0, 0, 0],[0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]])

#T = np.array([tpmStart, tpmOne, tpmTwo, tpmThree, tpmFour])
T = np.array([tmpA, tmpB])
print(T.shape)

# Reward Matrix:
# make a reward which gets worse for each step => goal minimize distance
# measure steps from start to storage place for each state:
# 1: 1, 2,3: 2, 4: 3 => for reward take negative values
# Action A, B in this case
#R = np.array([[-1, -2, -3, -3, 0],[0, -2, 0, 0, 0]])
# arrange shape
R = np.array([[1, 0],[1/2, 1/2],[1/3, 0], [1/3, 0], [0, 0]])
#R = np.zeros((2,5))
#R[0] = [-1, -2, -3, -3, 0] # action A
#R[1] = [0, -2, 0, 0, 0] # action B
print(R.shape)

# try model:
mdpWarehousePolicy = mdptoolbox.mdp.PolicyIteration(T, R, 0.3, max_iter=100)

# Run the MDP
mdpWarehousePolicy.run()

print('PolicyIteration:')
print(mdpWarehousePolicy.policy)
print(mdpWarehousePolicy.V)
print(mdpWarehousePolicy.iter)

(2, 5, 5)
(5, 2)
PolicyIteration:
(0, 0, 0, 0, 0)
(1.1896360520213731, 0.632120173404577, 0.4404005780152569, 0.4404005780152569, 0.3568908156064119)
1


In [20]:
# new attempt 2x2 warehouse:
# State is only the state of the warehouse => 4 fields a 7 states (empty = 0, 
# white_unstore = 1, red_unstore = 2, blue_unstore = 3, 
# white_store = 4, red_store = 5, blue_store = 6) (7^4 = 2401 states) 


# T has size (Action, State, State)
# Action is the position in the warehouse that is chosen
# 4 actions? (1,1)(1,2)(2,1)(2,2)
 

# R has size (State, Action)



[[0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.33333333 0.         0.33333333 0.         0.33333333 0.
  0.         0.         0.         0.        ]
 [0.         0.33333333 0.         0.33333333 0.         0.33333333
  0.         0.         0.         0.        ]
 [0.         0.         0.5        0.         0.         0.
  0.5        0.         0.         0.        ]
 [0.         0.33333333 0.         0.         0.         0.33333333
  0.         0.33333333 0.         0.        ]
 [0.         0.         0.25       0.         0.25       0.
  0.25       0.         0.25       0.        ]
 [0.         0.         0.         0.33333333 0.         0.33333333
  0.         0.         0.         0.33333333]
 [0.         0.         0.         0.         0.5        0.
  0.         0.         0.5        0.        ]
 [0.         0.         0.         0.         0.         0.33333333
  0.         0.33333333 0.         0.33333333]
 [0. 

In [ ]:
# create mdp
# just test
#mdp = mdptoolbox.mdp.MDP()